# A _statistical_ context-free grammar

**Idea:** generate a language in which sequences of symbols from a vocabulary are generated using a tree-based statistical algorithm.

### Algorithm

Given a vocabulary (or alphabet) of symbols $\mathcal{A} = \left\lbrace v_0, \ldots, v_{q-1}\right\rbrace$ ($q$ symbols in total):
- Start with a "root" symbol $a$ chosen uniformly from the vocabulary.
- Generate two children nodes, each containing a symbol, with the ordered pair of symbols $(b, c)$ in the children nodes chosen randomly according to a "transition tensor" (conditional probability) $p((b, c) | a) \equiv \rho_{abc}$. The tensor must obey the usual normalization of probabilities:
$$
\sum_{(b, c)} \rho_{abc} = 1\qquad \forall a\in\mathcal{A}\,.
$$
- From each children nodes generate another pair of children nodes according to the rule above, stopping after $K$ total iteration so as to form a tree with $K + 1$ levels (including the root) and with $2^K$ final children nodes ("leaves").
- Nodes (and thus symbols) are **ordered**: at each branching, the newly generated pairs of symbols keep the same relative ordering as the nodes that generated them (if a parent nodes are is to the left of another, the children of the first are to the left of those of the latter).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

### Transition tensor and nodes generation

In [ ]:
# Branching probabilities. a,b,c are letters in 0...q-1
# At each branching of the tree, if the letter is a, it generates 
# at the next level an ordered pair (b,c) with probability rho[a,b,c]
# By definition, for all a: sum_{b,c} rho[a,b,c]=1 
def calcrho_lognormal(q, sigma):
    """
    Generates a tensor with entries sampled from a log-normal
    distribution and normalizes it.
    """
    logrho=np.random.normal(0,size=(q,q,q))*sigma
    
    for a in np.arange(q):
        lrmax=np.max(logrho[a,:,:])
        logrho[a,:,:]=logrho[a,:,:]-lrmax
    rho=np.exp(logrho)

    # Normalize summing over the pairs (b, c) for each a (root).
    rho2=np.zeros((q,q))
    
    for a in np.arange(q):
        rho2=rho[a,:,:]
        sum2=np.sum(rho2)
        rho[a,:,:]=rho[a,:,:]/sum2
        
    return rho

# Conversion of b,c in [0,...,q-1]*[0,...,q-1] to nn in [0,...,q**2-1] and reciprocally
def calcnn(b,c):
    return b+q*c

def calcbc(nn):
    b=nn%q
    c=int((nn-b)/q)
    return b,c

#Broadcast: given a, generate b and c. 
def calcpsum(q,rho):
    """
    For each root node a, computes the cumulative sum of probabilities
    for children pairs (b, c).
    """
    p=np.zeros((q,q**2))
    psum=np.zeros((q,q**2))
    sumrule=np.zeros(q)

    # For each root a, computes the probabilities of children pair
    # (b, c) and stores them in p.
    for a in np.arange(q):
        for b in np.arange(q):
            for c in np.arange(q):
                nn=calcnn(b,c)

                # Probability of the ordered pair (b, c) given a.
                p[a,nn]=rho[a,b,c]
        sumrule=np.sum(p,axis=1)
    #print('sumrule',sumrule)

    # For each root a, computes the cumulative sum of the probabilities
    # of each children pair (b, c).
    for a in np.arange(q):
        psum[a,0]=p[a,0]
        for i in np.arange(q**2-1):
            j=i+1
            psum[a,j]=psum[a,j-1]+p[a,j]

    return psum

def genbc(q,a,psum):
    """
    Given a symbol a for a node, randomly generates an ordered pair
    of nodes (b, c) sampling from the distribution whose cumulative
    distribution sum is psum[a, :]. Essentially inverts the cumulative
    distribution function.
    """
    x=np.random.uniform(0,1)
    pp=np.zeros(q**2)
    pp=psum[a,:]
    for i in np.arange(q**2):
        #print('x,i,pp[i]',x,i,pp[i])
        if pp[i]>x:
            b,c=calcbc(i)
            return b,c
        
def xlog_qx(q,x):
    """
    Given q and x, computes x * log(x) / log(q) with the
    convention of returning 0 if x = 0.
    """
    if x==0 :
        return 0
    else:
        return x*np.log(x)/np.log(q)
    
def calc_rho_entrop(q,rho):
    ss=np.zeros(q)
    for a in np.arange(q):
        sum=0
        for b in np.arange(q):
            for c in np.arange(q):
                sum=sum-xlog_qx(q,rho[a,b,c])
        ss[a]=sum
    return np.mean(ss)/2

q=4
#u=1/100
#w=1/100#1/3-1/100
sigma=.01
rho = calcrho_lognormal(q,sigma)
psum=calcpsum(q,rho)
print('psum')
print(psum)
#print(rho)

for a in np.arange(q):
    print('a=',a,'rho=')
    print(rho[a,:,:])
    
print('entropy of rho, per output symbol',calc_rho_entrop(q,rho))

In [ ]:
# Test.
q = 4
sigma = .01

# Transition matrix (tensor).
rho = calcrho_lognormal(q, sigma)

# Check normalization.
print('Rho normalization (for each a):', rho.sum(axis=-1).sum(axis=-1))

# CDF(s) of the distribution(s) (for each a).
psum = calcpsum(q, rho)

# Plot some CDFs.
n_cdfs = 4

fig = plt.figure(figsize=(14, 6))

for i in np.random.choice(range(q), n_cdfs, replace=False):
    sns.lineplot(
        x=range(psum.shape[1]),
        y=psum[i, :]
    )

### Generation of the full tree

In [ ]:
#Broadcast in a binary tree with K+1 generations
# r=0 has one point
#r=1 has 2 points
# level r has 2**r points
# there are N=2**K leafs at level r=K
#x[r,i] is the letter at the i-th point of layer r, where i in [0,...2**r-1]
# In practice I define x as a matrix of size K+1, 2**K but at each layer r I use only the
# first 2**r-1 components
def gen_x(K,q,rho,root):
    """
    Generates a tree with K levels (after the root) given a vocabulary of
    size q, a transition tensor rho and a given root. Returns a matrix x of
    shape (K+1, N) (N = number of leaves = 2 ** K) where the first index refers
    to the level and the second to the nodes at that level. The matrix is
    rectangular despite the tree structure, therefore at level l, only the first
    2 ** l values of the second index must be considered (the others are all zero
    by convention).
    """
    N=2**K  # Number of leaves.

    x=np.zeros((K+1,N)).astype(int)

    # Assign the root.
    x[0,0]=root

    # Generate each level of the tree.
    for ir in np.arange(K):
        # ir refers to the parents' level, r to the children's level.
        r=ir+1
        jnew=0
        for j in np.arange(2**ir):
            a=x[ir,j]  # Parent symbol.
            
            b,c=genbc(q,a,psum)  # Generated children symbols.
            #print('r=',r,'jnew,jnew+1=',jnew,jnew+1)
            x[r,jnew]=b
            x[r,jnew+1]=c

            # Increase by 2 because each parent generates a pair of children.
            jnew=jnew+2
    #print('itry=',itry,'sequence obtained at generation K=',K)
    #print(x[K,:])
    return x

In [ ]:
# Small test of broadcast
K=3
N=2**K
root=0#np.random.randint(q)
ntry=1000  # Number of sequences to generate.
# xxx=np.zeros((K+1,N)).astype(int)
xleaf=np.zeros((ntry,N)).astype(int)
xf=np.zeros((ntry,K+1,N)).astype(int)
for itry in np.arange(ntry):
    xxx=gen_x(K,q,rho,root)
    xf[itry,:,:]=xxx
    xleaf[itry,:]=xf[itry,K,:]  # Leaves of each tree (observed sequences).
#print(xleaf)

In [ ]:
xm=np.mean(xleaf,axis=0)
print(xm)
plt.plot(xm)

### Belief propagation

In [ ]:
# Geometry
# Create a global index of the sites. There are in total 2**(K+1)-1 sites
# We use indsite[r,j]=n, and its reverse layer [n]=r  inlayer[n]=j
def calc_geom(K):
    ntot=2**(K+1)-1
    indsite=np.zeros((K+1,2**K),dtype=int)
    layer= np.zeros(ntot,dtype=int)
    inlayer= np.zeros(ntot,dtype=int)
    site=0
    for r in np.arange(K+1):
        for j in np.arange(2**r):
            indsite[r,j]=site
            layer[site]=r
            inlayer[site]=j
            site=site+1
        #print('r',r,'j',j,'site number',site)
# Create table of neighbours desc[n,0] and desc[n,1] are the two descendants above n
# asc[n] is the ascendant below n. I use the index -1 when there is no ascendant or descendant
    asc=-np.ones(ntot,dtype=int)
    desc=-np.ones((ntot,2),dtype=int)
    for n in np.arange(ntot):
        r=layer[n]
        j=inlayer[n]
        jkl=(np.floor(j/2)).astype(int)
        if r==0:
            asc[n]=-1
            desc[n,0]=indsite[1,0]
            desc[n,1]=indsite[1,1]
        if r==K:
            desc[n,0]=-1
            desc[n,1]=-1
            asc[n]=indsite[r-1,jkl]
        if (0<r) and (r<K):
            asc[n]=indsite[r-1,jkl]
            desc[n,0]=indsite[r+1,2*j]
            desc[n,1]=indsite[r+1,2*j+1]
        #print('n=',n,'r,j',r,j,'asc',asc[n],'desc',desc[n,0],desc[n,1])
    return indsite,layer,inlayer,asc,desc

In [ ]:
# Messages initialization
#Random
def initialize_messages(ntot,q):
    mup=np.random.uniform(0,1,size=(ntot,q))
    mdown=np.random.uniform(0,1,size=(ntot,q))
    mupsum=np.sum(mup,axis=1)
    mdownsum=np.sum(mdown,axis=1)
    for n in np.arange(ntot):
        for p in np.arange(q):
            mup[n,p]=mup[n,p]/mupsum[n]
            mdown[n,p]=mdown[n,p]/mdownsum[n]
            if layer[n]==0:
                mup[n,p]=1/q
            if layer[n]==K:
                mdown[n]=1/q
    return mup,mdown
#Random with constraint on the leaves and root
def initialize_messages_constraint(ntot,q,root,x,indsite):
    mup=np.random.uniform(0,1,size=(ntot,q))
    mdown=np.random.uniform(0,1,size=(ntot,q))
    mupsum=np.sum(mup,axis=1)
    mdownsum=np.sum(mdown,axis=1)
    for n in np.arange(ntot):
        for p in np.arange(q):
            mup[n,p]=mup[n,p]/mupsum[n]
            mdown[n,p]=mdown[n,p]/mdownsum[n]
            if layer[n]==0:
                mup[n,p]=1/q
            if layer[n]==K:
                mdown[n]=1/q
    for p in np.arange(q):
        mup[0,p]=0
    mup[0,root]=1
    for j in np.arange(2**K):
        n=indsite[K,j]
        for p in np.arange(q):
            mdown[n,p]=0
        mdown[n,x[j]]=1        
    return mup,mdown


In [ ]:
#Messages update
def calc_mout(q,inup,inleft,inright,rho):
    outdown=np.zeros(q)
    outleft=np.zeros(q)
    outright=np.zeros(q)
    for jdown in np.arange(q):
        sumt=0
        for jleft in np.arange(q):
            for jright in np.arange(q):
                sumt=sumt+inleft[jleft]*inright[jright]*rho[jdown,jleft,jright]
        outdown[jdown]=sumt
    sm=np.sum(outdown)
    outdown=outdown/sm
    for jleft in np.arange(q):
        sumt=0
        for jup in np.arange(q):
            for jright in np.arange(q):
                sumt=sumt+inup[jup]*inright[jright]*rho[jup,jleft,jright]
        outleft[jleft]=sumt
    sm=np.sum(outleft)
    outleft=outleft/sm
    for jright in np.arange(q):
        sumt=0
        for jup in np.arange(q):
            for jleft in np.arange(q):
                sumt=sumt+inup[jup]*inleft[jleft]*rho[jup,jleft,jright]
        outright[jright]=sumt
    sm=np.sum(outright)
    outright=outright/sm
    return outdown,outleft,outright
        
def update_messages(ntot,q,mup,mdown,asc,desc,rho):
    #print('check init update mess')
    #print('ntot=',ntot,'q=',q,np.shape(mup),np.shape(mdown),np.shape(asc),np.shape(desc))
    inleft=np.zeros(q)
    inright=np.zeros(q)
    inup=np.zeros(q)
    mupnew=np.copy(mup)
    mdownnew=np.copy(mdown)
    #Here to be done these loops should be parallelized 
    for r in np.arange(K): 
        for j in np.arange(2**r):
                n=indsite[r,j]# This is also the index of the factor node above the variable
                inup=mup[n,:]
                #print('check',n,asc[n],desc[n,0],desc[n,1])
                inleft=mdown[desc[n,0],:]
                inright=mdown[desc[n,1],:]
                outdown,outleft,outright=calc_mout(q,inup,inleft,inright,rho)
                mdownnew[n,:]=outdown
                mupnew[desc[n,0],:]=outleft
                mupnew[desc[n,1],:]=outright
    return mupnew,mdownnew
#Run BP until convergence
def BP_iter(ntot,q,mupinit,mdowninit,asc,desc,rho):
    mup=np.copy(mupinit)
    mdown=np.copy(mdowninit)
    for iter in np.arange(2*K+5):
        mupnew,mdownnew=update_messages(ntot,q,mup,mdown,asc,desc,rho)
        errmess=np.sum((mupnew-mup)**2)+np.sum((mdownnew-mdown)**2)
        mup=np.copy(mupnew)
        mdown=np.copy(mdownnew)
        if(errmess<1.e-6):
            break
    return mup,mdown
#Compute the free entropy of the tree with constraints x_0=root
# and x_Boundary=x. Returns minus the free entropy divided by the size
# of the boundary
def calc_entrop_messages(mup,mdown,rho,ntot,q,K,desc,indsite):
    phi_fact=0
    for r in np.arange(K):
        for j in np.arange(2**r):
            n=indsite[r,j]
            sumtot=0
            for p1 in np.arange(q):
                for p2 in np.arange(q):
                    for p3 in np.arange(q):
                        sumtot=sumtot+mdown[desc[n,0],p1]*mdown[desc[n,1],p2]*mup[n,p3]*rho[p3,p1,p2]
            #print('test',r,j,n,sumtot)
            phi_fact=phi_fact+np.log(sumtot)/np.log(q)
    phi_var=0
    for r in np.arange(1,K):
        for j in np.arange(2**r):
            n=indsite[r,j]
            sumtot=0
            for p1 in np.arange(q):
                sumtot=sumtot+mup[n,p1]*mdown[n,p1]
            phi_var=phi_var+np.log(sumtot)/np.log(q)
    return -(phi_fact-phi_var)/2**K

In [ ]:
#Main

#Define the geometry
K=8
N=2**K
ntot=2**(K+1)-1
indsite,layer,inlayer,asc,desc=calc_geom(K)
#Compute the transition tensor M
q=3
numsigma=12
numMat=20
av_ent=np.zeros((numsigma,numMat))
ste_ent=np.zeros((numsigma,numMat))
rho_ent=np.zeros((numsigma,numMat))
sigmag=np.zeros(numsigma)
for isigma in np.arange(numsigma):
    sigma=.2+(isigma)/1.5
    sigmag[isigma]=sigma
    for iMat in np.arange(numMat):
        rho = calcrho_lognormal(q,sigma)
        psum=calcpsum(q,rho)
        rho_ent[isigma,iMat]=calc_rho_entrop(q,rho)
#Broadcast
        root=0#np.random.randint(q)
        ntry=3
        xxx=np.zeros((K+1,N)).astype(int)
        xleaf=np.zeros((ntry,N)).astype(int)
        xf=np.zeros((ntry,K+1,N)).astype(int)
        for itry in np.arange(ntry):
            xxx=gen_x(K,q,rho,root)
            xf[itry,:,:]=xxx
            xleaf[itry,:]=xf[itry,K,:]
#Solve BP and compute the entropy per variable
        etm=np.zeros(ntry)
        for itry in np.arange(ntry):
            x=xleaf[itry,:]
            mupinit,mdowninit=initialize_messages_constraint(ntot,q,root,x,indsite)
            mup,mdown=BP_iter(ntot,q,mupinit,mdowninit,asc,desc,rho)
            entropmes=calc_entrop_messages(mup,mdown,rho,ntot,q,K,desc,indsite)
            etm[itry]=entropmes
        print('sigma',sigma,'iMat',iMat,'itry=',itry,'entropmes=',np.mean(etm))
        av_ent[isigma,iMat]=np.mean(etm)
        ste_ent[isigma,iMat]=np.std(etm)/np.sqrt(ntry)
print('done')

sigma 4.2 iMat 7 itry= 2 entropmes= 0.5202833110493835
sigma 4.2 iMat 8 itry= 2 entropmes= 0.4793104536098775
sigma 4.2 iMat 9 itry= 2 entropmes= 0.5276173864477265
sigma 4.2 iMat 10 itry= 2 entropmes= 0.5580363481286185
sigma 4.2 iMat 11 itry= 2 entropmes= 0.5679834392292948
sigma 4.2 iMat 12 itry= 2 entropmes= 0.5869279386517549
sigma 4.2 iMat 13 itry= 2 entropmes= 0.7121443527188441
sigma 4.2 iMat 14 itry= 2 entropmes= 0.634136287174929
sigma 4.2 iMat 15 itry= 2 entropmes= 0.8406078216958816
sigma 4.2 iMat 16 itry= 2 entropmes= 0.6839708616378958
sigma 4.2 iMat 17 itry= 2 entropmes= 0.7557769107220094
sigma 4.2 iMat 18 itry= 2 entropmes= 0.6801563392886466
sigma 4.2 iMat 19 itry= 2 entropmes= 0.14767511987383714
sigma 4.866666666666667 iMat 0 itry= 2 entropmes= 0.5701846873512457
sigma 4.866666666666667 iMat 1 itry= 2 entropmes= 0.7643056254241221
sigma 4.866666666666667 iMat 2 itry= 2 entropmes= 0.42844263500902174
sigma 4.866666666666667 iMat 3 itry= 2 entropmes= 0.301131290854521

In [ ]:
for isigma in np.arange(numsigma):
    print('sigma=',sigmag[isigma],'ent=',av_ent[isigma,:],'+-',ste_ent[isigma,:])

In [ ]:
#plt.plot(sigmag,np.mean(av_ent,axis=1))
fig, ax = plt.subplots()
ax.errorbar(sigmag,np.mean(av_ent,axis=1),np.std(av_ent,axis=1)/np.sqrt(numMat-1))
plt.xlabel('sigma')
plt.ylabel('H')
plt.savefig('entropy_vs_sigma.pdf')
plt.show()

In [ ]:
for isi in np.arange(numsigma):
    print(isi,sigmag[isi])

In [ ]:
isigma=10
plt.plot(rho_ent[isigma,:],av_ent[isigma,:],'b*')
plt.xlabel('H rho')
plt.ylabel('H boundary')
plt.text(.05,.6,'sigma=6.87')
plt.savefig('entropyB vs entropy rho.pdf')
plt.show()